1. Data Loading and Cleaning

· Load the dataset into a Pandas DataFrame.

· Identify and handle missing values appropriately (e.g., using imputation techniques).

· Convert categorical variables into numerical format (e.g., label encoding or one-hot encoding).

· Scale numerical features if necessary (e.g., StandardScaler, MinMaxScaler) and explain why.

In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

df = pd.read_csv("./credit_approval_anonymized.csv")

df.replace("", np.nan, inplace=True)
df.dropna()

categorical_cols = ['A1', 'A4', 'A5', 'A6', 'A7', 'A8', 'A10', 'A9', 'A12', 'A13']
numerical_cols = ['A2', 'A3', 'A11', 'A14', 'A15']


target_map = {'+': 1, '-': 0}
df['A16'] = df['A16'].map(target_map)

encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])


scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

### Clarification to why we are using StandardScaling